In [1]:
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import seaborn as sns
# pd.set_option('display.max_rows', None)
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel
from transformers import AdamW
import torch.nn as nn
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm

In [5]:
df = pd.read_csv("/content/drive/MyDrive/University/4-2/정보기술학회/df_merge.csv", encoding = "utf-8")

In [6]:
df.head()

,성별코드,연령대코드,요양개시일자,서식코드,진료과목코드,주상병코드,부상병코드,요양일수,입내원일수,심결가산율,심결요양급여비용총액,심결본인부담금,심결보험자부담금,총처방일수,주상병_한글명,부상병_한글명
0,2,14,2023-02-02,3,1,J20,J30,1,1,0.15,12380,1500,10880,4,급성 기관지염,혈관운동성 및 알레르기성 비염
1,1,14,2023-02-06,3,1,J20,J30,1,1,0.15,12380,1500,10880,5,급성 기관지염,혈관운동성 및 알레르기성 비염
2,1,14,2023-02-02,3,1,J20,J30,1,1,0.15,12380,1500,10880,3,급성 기관지염,혈관운동성 및 알레르기성 비염
3,2,14,2023-02-28,3,1,J20,J30,1,1,0.15,12380,1500,10880,3,급성 기관지염,혈관운동성 및 알레르기성 비염
4,1,14,2023-02-15,3,1,J20,J30,1,1,0.15,12380,1500,10880,2,급성 기관지염,혈관운동성 및 알레르기성 비염


In [7]:
# 진료과목코드와 실제 진료과 매핑
mapping = {
    0: '일반의', 1: '내과', 2: '신경과', 3: '정신과', 4: '외과', 5: '정형외과', 6: '신경외과',
    7: '흉부외과', 8: '성형외과', 9: '마취통증의학과', 10: '산부인과', 11: '소아청소년과',
    12: '안과', 13: '이비인후과', 14: '피부과', 15: '비뇨기과', 16: '영상의학과',
    17: '방사선 종양학과', 18: '병리과', 19: '진단검사의학과', 20: '결핵과',
    21: '재활의학과', 22: '핵의학과', 23: '가정의학과', 24: '응급의학과',
    25: '산업의학과', 26: '예방의학과', 50: '구강악안면외과', 51: '치과보철과',
    52: '치과교정과', 53: '소아치과', 54: '치주과', 55: '치과보존과',
    56: '구강내과', 57: '구강악안면방사선과', 58: '구강병리과',
    59: '예방치과', 80: '한방내과', 81: '한방부인과',
    82: '한방소아과', 83: '한방안·이비인후·피부 과',
    84: '한방신경정신 과', 85: '침구 과',
    86: '한방재활의학 과',87:'사상체질 과',
    88:'한방응급 과','ZZ':'Nan'
}

# 진료코드를 실제 진료 과로 변환
df['진료과'] = df['진료과목코드'].map(mapping)

# 변환된 결과 출력
df.head()

,성별코드,연령대코드,요양개시일자,서식코드,진료과목코드,주상병코드,부상병코드,요양일수,입내원일수,심결가산율,심결요양급여비용총액,심결본인부담금,심결보험자부담금,총처방일수,주상병_한글명,부상병_한글명,진료과
0,2,14,2023-02-02,3,1,J20,J30,1,1,0.15,12380,1500,10880,4,급성 기관지염,혈관운동성 및 알레르기성 비염,내과
1,1,14,2023-02-06,3,1,J20,J30,1,1,0.15,12380,1500,10880,5,급성 기관지염,혈관운동성 및 알레르기성 비염,내과
2,1,14,2023-02-02,3,1,J20,J30,1,1,0.15,12380,1500,10880,3,급성 기관지염,혈관운동성 및 알레르기성 비염,내과
3,2,14,2023-02-28,3,1,J20,J30,1,1,0.15,12380,1500,10880,3,급성 기관지염,혈관운동성 및 알레르기성 비염,내과
4,1,14,2023-02-15,3,1,J20,J30,1,1,0.15,12380,1500,10880,2,급성 기관지염,혈관운동성 및 알레르기성 비염,내과


In [11]:
df['총처방일수'].value_counts()

,count
총처방일수,
0,2875754
3,2178223
5,1167041
7,1087326
4,1020765
...,...
750,1
253,1
570,1


### EDA

In [4]:
df.head()

,성별코드,연령대코드,요양개시일자,서식코드,진료과목코드,주상병코드,부상병코드,요양일수,입내원일수,심결가산율,심결요양급여비용총액,심결본인부담금,심결보험자부담금,총처방일수,주상병_한글명,부상병_한글명
0,2,14,2023-02-02,3,1,J20,J30,1,1,0.15,12380,1500,10880,4,급성 기관지염,혈관운동성 및 알레르기성 비염
1,1,14,2023-02-06,3,1,J20,J30,1,1,0.15,12380,1500,10880,5,급성 기관지염,혈관운동성 및 알레르기성 비염
2,1,14,2023-02-02,3,1,J20,J30,1,1,0.15,12380,1500,10880,3,급성 기관지염,혈관운동성 및 알레르기성 비염
3,2,14,2023-02-28,3,1,J20,J30,1,1,0.15,12380,1500,10880,3,급성 기관지염,혈관운동성 및 알레르기성 비염
4,1,14,2023-02-15,3,1,J20,J30,1,1,0.15,12380,1500,10880,2,급성 기관지염,혈관운동성 및 알레르기성 비염


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12150274 entries, 0 to 12150273
Data columns (total 16 columns):
 #   Column      Dtype  
---  ------      -----  
 0   성별코드        int64  
 1   연령대코드       int64  
 2   요양개시일자      object 
 3   서식코드        int64  
 4   진료과목코드      int64  
 5   주상병코드       object 
 6   부상병코드       object 
 7   요양일수        int64  
 8   입내원일수       int64  
 9   심결가산율       float64
 10  심결요양급여비용총액  int64  
 11  심결본인부담금     int64  
 12  심결보험자부담금    int64  
 13  총처방일수       int64  
 14  주상병_한글명     object 
 15  부상병_한글명     object 
dtypes: float64(1), int64(10), object(5)
memory usage: 1.4+ GB


In [5]:
df.shape

(12150274, 16)

In [7]:
df_objetc = df.select_dtypes(include = 'object')
df_objetc.describe()

,요양개시일자,주상병코드,부상병코드,주상병_한글명,부상병_한글명
count,12150274,12150274,12150274,12150274,12150274
unique,365,1235,1385,1235,1385
top,2023-12-26,J20,K29,급성 기관지염,위염 및 십이지장염
freq,61654,1204901,1143006,1204901,1143006


### Modeling


In [12]:
# GPU 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("============")
print(device)
print("============")

# 필요한 컬럼 선택 및 결측치 처리
selected_columns = ['성별코드', '연령대코드', '진료과목코드', '주상병코드', '주상병_한글명', '요양일수', '입내원일수', '총처방일수']
df_selected = df[selected_columns].dropna()

# 카테고리 인코딩 (성별코드, 연령대코드, 진료과목코드)
df_selected['성별코드'] = df_selected['성별코드'].astype('category').cat.codes
df_selected['연령대코드'] = df_selected['연령대코드'].astype('category').cat.codes
df_selected['진료과목코드'] = df_selected['진료과목코드'].astype('category').cat.codes

# 상위 100개의 주상병코드를 선택하고 나머지는 '기타'로 처리
top_n = 100
top_labels = df_selected['주상병코드'].value_counts().index[:top_n]
df_selected['주상병코드'] = df_selected['주상병코드'].apply(lambda x: x if x in top_labels else '기타')

# 라벨 인코딩 (주상병코드)
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df_selected['주상병코드_encoded'] = label_encoder.fit_transform(df_selected['주상병코드'])

# feature와 target 설정 (구조적 데이터와 텍스트 데이터 분리)
X_structured = df_selected[['성별코드', '연령대코드', '진료과목코드']]
X_text = df_selected['주상병_한글명']
Y = df_selected['주상병코드_encoded']

# 데이터 나누기 (80% 훈련, 20% 테스트)
X_train_structured, X_test_structured, X_train_text, X_test_text, Y_train, Y_test = train_test_split(
    X_structured, X_text, Y, test_size=0.3, random_state=42)

cuda


In [18]:
# BERT Tokenizer 설정
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

# Custom Dataset 클래스 정의 (BERT 입력을 위한 Dataset)
class CustomDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts.iloc[idx]
        label = self.labels.iloc[idx]
        encoding = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=128,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

# 데이터 로더 생성
train_dataset = CustomDataset(X_train_text, Y_train)
test_dataset = CustomDataset(X_test_text, Y_test)

train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=256)

In [19]:
# BERT 기반 모델 정의 (간단한 분류기)
class BERTClassifier(nn.Module):
    def __init__(self):
        super(BERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-multilingual-cased')
        self.dropout = nn.Dropout(0.3)
        self.out = nn.Linear(self.bert.config.hidden_size, len(Y.unique()))  # 주상병코드의 고유값 개수만큼 출력

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs[1]
        output = self.dropout(pooled_output)
        return self.out(output)


# 모델 초기화 및 설정
model_bert = BERTClassifier().to(device)
optimizer = AdamW(model_bert.parameters(), lr=2e-5)
criterion = nn.CrossEntropyLoss()

# BERT 모델 학습 함수 정의
def train_bert_model(model, data_loader):
    model.train()
    total_loss = 0
    for batch in tqdm(data_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    return total_loss / len(data_loader)

# BERT 모델 평가 함수 정의
def evaluate_bert_model(model, data_loader):
    model.eval()
    predictions, true_labels = [], []
    with torch.no_grad():
        for batch in tqdm(data_loader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs, dim=1)

            predictions.extend(preds.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(true_labels, predictions)
    report = classification_report(true_labels, predictions)

    return accuracy, report

# BERT 학습 및 평가 실행 (Epoch 3회 반복)
for epoch in range(3):
    print(f'Epoch {epoch + 1}/3')
    train_loss_bert = train_bert_model(model_bert, train_loader)
    print(f'Train Loss: {train_loss_bert}')

accuracy_bert, report_bert = evaluate_bert_model(model_bert, test_loader)
print(f'BERT Model Accuracy: {accuracy_bert}')
print(report_bert)

Epoch 1/3


  0%|          | 102/33224 [01:53<10:13:59,  1.11s/it]


KeyboardInterrupt: 